In [1]:
using Pkg
Pkg.activate("EnvFerrite")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


In [2]:
using TypedPolynomials
@polyvar x y

(x, y)

In [3]:
p = 2x*y^2 +y -2x
q = 3x*y^3 - 4x + 4

4 - 4x + 3xy³

In [4]:
function nabladotnabla(a,b)
    diff_a = differentiate(a,(x,y))
    diff_b = differentiate(b,(x,y))
    return diff_a[1]*diff_b[1]+diff_b[2]*diff_b[2]
end

nabladotnabla (generic function with 1 method)

In [5]:
r = nabladotnabla(p,q)

8 - 8y² - 6y³ + 6y⁵ + 81x²y⁴

In [6]:
using Ferrite
using SparseArrays
using LinearAlgebra
using IterativeSolvers


In [7]:
grid = generate_grid(Quadrilateral, (16, 16));
dim = Ferrite.getspatialdim(grid)
order = 1

ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(2)
cellvalues = CellValues(qr, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)

DofHandler{2, Grid{2, Quadrilateral, Float64}}
  Fields:
    :u, Lagrange{RefQuadrilateral, 1}()
  Dofs per cell: 4
  Total dofs: 289

In [8]:
# for that we also gonna assemble the Mass Matrix:
# Mass Matrix  ∫(u*v)dΩ
function assemble_M(cellvalues::CellValues,dh::DofHandler)
    M = allocate_matrix(dh)
    n_basefuncs = getnbasefunctions(cellvalues)
    Me = zeros(n_basefuncs, n_basefuncs)
    assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Me, 0)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)      
            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    φⱼ = shape_value(cellvalues, q_point, j)
                    Me[i,j] += φᵢ * φⱼ * dΩ
                end
            end
        end
        assemble!(assembler, celldofs(cell), Me)
    end
    return M, cholesky(M)
end   
M,MC = assemble_M(cellvalues,dh)

(sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  270, 271, 272, 287, 288, 289, 271, 272, 288, 289], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  288, 288, 288, 288, 288, 288, 289, 289, 289, 289], [0.001736111111111111, 0.0008680555555555556, 0.00043402777777777775, 0.0008680555555555555, 0.0008680555555555556, 0.0034722222222222225, 0.001736111111111111, 0.00043402777777777775, 0.0008680555555555556, 0.00043402777777777775  …  0.00043402777777777775, 0.001736111111111111, 0.00043402777777777765, 0.0008680555555555553, 0.003472222222222221, 0.0008680555555555553, 0.00043402777777777765, 0.0008680555555555553, 0.0008680555555555553, 0.0017361111111111106], 289, 289), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  3815
nnz:     3815
success: true
)

In [9]:
# now we gonna project conductivity unto a vector:
function assemble_function_vector(cellvalues::CellValues, dh::DofHandler, f, M)
    F = zeros(ndofs(dh))
    n_basefuncs = getnbasefunctions(cellvalues)
    Fe = zeros(n_basefuncs)
    cdofs = zeros(Int, n_basefuncs)

    for cell in CellIterator(dh)
        fill!(Fe, 0.0)
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        cdofs = celldofs(cell)
        for q in 1:getnquadpoints(cellvalues)
            x_q = spatial_coordinate(cellvalues, q, coords)
            f_val = f(x_q)
            dΩ = getdetJdV(cellvalues, q)

            for i in 1:n_basefuncs
                Fe[i] += f_val * shape_value(cellvalues, q, i) * dΩ
            end
        end  
        assemble!(F, cdofs,Fe)
    end
    return M \ F

end

assemble_function_vector (generic function with 1 method)

In [10]:
function get_func(p)
    (z) -> p(x=>z[1],y=>z[2])
end

get_func (generic function with 1 method)

In [11]:
p_func = get_func(p)
q_func = get_func(q)
r_func = get_func(r)

#11 (generic function with 1 method)

In [12]:
p_vec = assemble_function_vector(cellvalues, dh, p_func, MC)
q_vec = assemble_function_vector(cellvalues, dh, q_func, MC)
r_vec = assemble_function_vector(cellvalues, dh, r_func, MC)

289-element Vector{Float64}:
 79.70860712489336
 61.00620508727424
 38.381961652895015
 49.27970789351367
 44.797456654670974
 28.937248244358834
 31.082361827083606
 20.945567667905145
 19.860920604512106
 14.406919923533977
  ⋮
 -0.28563403188767583
  0.9611927706202666
  4.7016731781440955
 10.935807190683803
 19.663594808239402
 30.885036030810898
 44.60013085839825
 60.80887929100154
 79.51128132862067

now we can write an assembler for $\nabla(a)\cdot\nabla(b)$

In [13]:
# Write assembler here: for ∇(a)⋅∇(b)
function calculate_bilinear_map_rhs(a::AbstractVector,b::AbstractVector, cellvalues::CellValues,dh::DofHandler, M)
    n = ndofs(dh)
    rhs = zeros(n)
    n_basefuncs = getnbasefunctions(cellvalues)
    qpoints = getnquadpoints(cellvalues)
    ae = zeros(n_basefuncs)
    be = zeros(n_basefuncs)
    re = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        dofs = celldofs(cell)
        reinit!(cellvalues,cell)
        for q in 1:qpoints 
            dΩ = getdetJdV(cellvalues, q)
            for i in 1:n_basefuncs
                ∇ϕᵢ = shape_gradient(cellvalues, q, i)    
                ϕᵢ = shape_value(cellvalues, q, i)
                for j in 1:n_basefuncs
                    ∇ϕⱼ = shape_gradient(cellvalues, q, j)
                    re[i] = ae[i]*be[j]*(∇ϕᵢ⋅∇ϕⱼ)* dΩ
                end
            end
        end
        assemble!(rhs, dofs, re)
    end
    return M \ rhs
end

calculate_bilinear_map_rhs (generic function with 1 method)

In [14]:
r_test = calculate_bilinear_map_rhs(p_vec,q_vec, cellvalues,dh,MC)

289-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
@assert norm(r_vec - r_test) < 10.0

AssertionError: AssertionError: norm(r_vec - r_test) < 10.0